# Max Min Ant System

In [ ]:
graphe1 = {0: {'cluster': 0,
               'neighbours': {1: [77], 2: [82]},
               'x': 0,
               'y': 0},
           1: {'cluster': 0,
               'neighbours': {0: [77], 2: [82], 3: [19]},
               'x': -43,
               'y': -64},
           2: {'cluster': 0,
               'neighbours': {0: [82], 1: [82], 3: [64]},
               'x': -60,
               'y': 17},
           3: {'cluster': 0,
               'neighbours': {1: [19], 2: [64], 4: [96], 5: [70], 6: [88]},
               'x': -27,
               'y': -53},
           4: {'cluster': 0,
               'neighbours': {3: [96], 5: [70], 6: [52], 7: [44]},
               'x': 65,
               'y': -83},
           5: {'cluster': 0,
               'neighbours': {3: [70], 6: [45], 7: [50]},
               'x': 43,
               'y': -43},
           6: {'cluster': 0,
               'neighbours': {3: [88], 7: [93], 8: [7]},
               'x': 45,
               'y': -104},
           7: {'cluster': 0,
               'neighbours': {6: [93], 8: [43], 9: [57]},
               'x': -48,
               'y': -91},
           8: {'cluster': 0,
               'neighbours': {6: [7], 9: [34], 10: [120], 11: [37]},
               'x': 43,
               'y': -111},
           9: {'cluster': 0,
               'neighbours': {8: [34], 10: [67], 11: [94]},
               'x': 16,
               'y': -89},
           10: {'cluster': 0,
                'neighbours': {8: [120], 11: [89], 12: [89]},
                'x': -52,
                'y': -185},
           11: {'cluster': 0,
                'neighbours': {8: [37], 12: [47]},
                'x': 65,
                'y': -141},
           12: {'cluster': 0,
                'neighbours': {11: [47], 13: [67]},
                'x': 40,
                'y': -181},
           13: {'cluster': 0,
                'neighbours': {12: [67], 14: [112]},
                'x': 90,
                'y': -227},
           14: {'cluster': 0,
                'neighbours': {13: [112], 15: [62]},
                'x': 31,
                'y': -131},
           15: {'cluster': 0,
                'neighbours': {14: [62]},
                'x': 61,
                'y': -76}}

## Algorithme Principal

In [ ]:
import random

N_ANTS = 100
ALPHA = 1.0 # Influence de la trace de phéromone (+ -> fourmis suivent plus la trace)
BETA = 5.0 # Influence de la visibilité (+ -> chemin le plus court)
EVAPORATION_RATE = 1 # rho
TAU_MIN = 0.01 # Phéromone minimale
TAU_MAX = 2.0 # Phéromone maximale
STAGNATION_MAX = 500 # Nombre d’itérations sans amélioration avant stagnation

def distance(graph, a, b):
    if b in graph[a]['neighbours']:
        return graph[a]['neighbours'][b][0]  # On utilise le poids comme distance
    return float('inf')


def initialize_pheromones(graph):
    pheromones = {}
    for vertex in graph:
        for neighbour in graph[vertex]['neighbours']:
                pheromones[(vertex, neighbour)] = TAU_MAX
                pheromones[(neighbour, vertex)] = TAU_MAX
    return pheromones


def transition_probability(current, neighbours, pheromones):
    numerators = {}
    total = 0.0

    for neighbour in neighbours:
        weight = min(neighbours[neighbour])
        tau = pheromones.get((current, neighbour, weight), TAU_MIN)
        eta = 1.0 / weight  # heuristic information
        numerators[neighbour] = (tau ** ALPHA) * (eta ** BETA)
        total += numerators[neighbour]

    if total == 0:
        # If all probabilities are 0, distribute equally
        return {n: 1.0 / len(neighbours) for n in neighbours}

    return {n: numerators[n] / total for n in numerators}


def next_vertex(probas):
    r = random.random()
    cumulative = 0.0
    for vertex, prob in probas.items():
        cumulative += prob
        if r <= cumulative:
            return vertex
    return list(probas.keys())[-1]  # fallback


def path_cost(path, graph):
    cost = 0
    for i in range(len(path)-1):
        u = path[i]
        v = path[i+1]
        cost += min(graph[u]['neighbours'][v])
    return cost


def max_min_ant_system(graph, starting_vertex):
    pheromones = initialize_pheromones(graph)
    best_path = None
    best_cost = float('inf')
    stagnation = 0

    while True:
        solutions = []

        # Each ant builds a solution
        for _ in range(N_ANTS):
            path = [starting_vertex]
            current = starting_vertex
            visited = set(path)

            # Build path until all vertices are visited
            while len(visited) < len(graph):
                # Get unvisited neighbors
                neighbors = {
                    n: graph[current]['neighbours'][n]
                    for n in graph[current]['neighbours']
                    if n not in visited
                }

                if not neighbors:
                    break

                # Choose next vertex
                probabilities = transition_probability(current, neighbors, pheromones)
                next_node = next_vertex(probabilities)
                path.append(next_node)
                visited.add(next_node)
                current = next_node

                if len(path) == len(graph):  # Chemin valide (visite tous les sommets)
                    cost = path_cost(path, graph)
                    solutions.append((path, cost))

        # If no solution found, continue
        if not solutions:
            continue

        iteration_best_path, iteration_best_cost = min(solutions, key=lambda x: x[1])

        if iteration_best_cost < best_cost:
            best_path = iteration_best_path
            best_cost = iteration_best_cost
            print("New solution found : ", best_path, "cost : ", best_cost)
            stagnation = 0
        else:
            stagnation += 1

        if stagnation >= STAGNATION_MAX:
            break

        # Évaporation
        for edge in pheromones:
            pheromones[edge] = max(pheromones[edge] * (1 - EVAPORATION_RATE), TAU_MIN)

        # Renforcement du chemin optimal
        delta_tau = 1.0 / best_cost
        for i in range(len(best_path) - 1):
            u = best_path[i]
            v = best_path[i + 1]
            for edge in [(u, v), (v, u)]:
                pheromones[edge] = min(pheromones.get(edge, TAU_MIN) + delta_tau, TAU_MAX)

    if best_path:
        best_path.append(best_path[0])  # cycle hamiltonien

    return best_path, best_cost

## Rendre le graphe complet avec A*

In [ ]:
import copy

def heuristic(graph, a, b):
    weights = [w[0] for w in graph[a]['neighbours'].values()]
    return min(weights) if weights else 0

def a_star_weighted(graph, start, goal):
    open_set = [(0, start)]  # liste de tuples (f_score, sommet)
    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0

    while open_set:
        # Trie pour récupérer le noeud avec le plus petit f_score
        open_set.sort(key=lambda x: x[0])
        _, current = open_set.pop(0)

        if current == goal:
            path = []
            total_cost = g_score[goal]
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path, total_cost

        for neighbour, weights in graph[current]['neighbours'].items():
            weight = weights[0]
            tentative_g_score = g_score[current] + weight

            if tentative_g_score < g_score[neighbour]:
                came_from[neighbour] = current
                g_score[neighbour] = tentative_g_score
                f_score = tentative_g_score + heuristic(graph, neighbour, goal)

                # Mise à jour dans open_set : on évite les doublons
                found = False
                for i in range(len(open_set)):
                    if open_set[i][1] == neighbour:
                        if f_score < open_set[i][0]:
                            open_set[i] = (f_score, neighbour)
                        found = True
                        break
                if not found:
                    open_set.append((f_score, neighbour))
    return None, float('inf')


def build_complete_graph(graph):
    graph2 = copy.deepcopy(graph)
    correspondance = {}
    for u in graph:
        for v in graph:
            if u == v or v in graph[u]['neighbours']:
                continue

            path, cost = a_star_weighted(graph, u, v)

            if path and cost < float('inf'):
                correspondance[u,v] = path
                graph2[u]['neighbours'][v] = [cost]
                graph2[v]['neighbours'][u] = [cost]

    return correspondance, graph2

def expand_path_with_mapping(path, edge_mapping):
    """Étend les arêtes du chemin selon le dictionnaire edge_mapping."""
    expanded_path = []
    for i in range(len(path) - 1):
        u, v = path[i], path[i + 1]
        if (u, v) in edge_mapping:
            segment = edge_mapping[(u, v)]
        elif (v, u) in edge_mapping:
            segment = edge_mapping[(v, u)][::-1]  # inverser si chemin dans l'autre sens
        else:
            segment = [u, v]

        if expanded_path:
            # Évite les doublons entre segments consécutifs
            expanded_path.extend(segment[1:])
        else:
            expanded_path.extend(segment)

    return expanded_path

## Test de l'algorithme

In [ ]:
correspondance, newGraph = build_complete_graph(graphe1)

best_path, best_cost = max_min_ant_system(newGraph, 0)
expanded_path = expand_path_with_mapping(best_path, correspondance)
print("Meilleur chemin :", expanded_path)
print("Coût total :", best_cost)